## Burned area

In [ ]:
workflow = dict([('id', 'burned-area'),
                ('label', 'Burned area delineation'),
                ('doc', 'Burned area delineation using two techniques')])


In [30]:
pre_event = dict([('id', 'pre_event'), 
                  ('label', 'Pre-event product for burned area delineation'),
                  ('doc', 'Pre-event product for burned area delineation'),
                  ('value', ''), 
                  ('stac:collection', 'pre_event'),
                  ('stac:href', '/workspace/nbr/stac-results/catalog.json'),
                  ('type', 'Directory')])

In [31]:
post_event = dict([('id', 'post_event'), 
                  ('label', 'Post-event product for burned area delineation'),
                  ('doc', 'Post-event product for burned area delineation'),
                  ('value', ''), 
                  ('stac:collection', 'post_event'),
                  ('stac:href', '/workspace/nbr/stac-results/catalog.json'),
                  ('type', 'Directory')])

In [32]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')]) 

In [33]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')]) 

In [34]:
aoi = dict([('id', 'aoi'), 
              ('label', 'Area of interest'),
              ('doc', 'Area of interest in WKT'),
              ('value', 'POLYGON((136.508 -36.108,136.508 -35.654,137.178 -35.654,137.178 -36.108,136.508 -36.108))'), 
              ('type', 'string')])

### Vegetation indexes

NBR = (NIR - SWIR22) / (NIR + SWIR22)
NDVI = (NIR - Red) / (NIR + Red)
NDWI = (NIR - SWIR16) / (NIR + SWIR16)

In [35]:
bands = [{'name': 'NBR',
          'common_name': 'nbr'}, 
         {'name': 'NDVI',
          'common_name': 'ndvi'},
         {'name': 'NDWI',
          'common_name': 'ndwi'},
        ]

In [36]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import Catalog, Collection, EOItem, MediaType, EOAsset, CatalogType
from time import sleep
from shapely.wkt import loads

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_nbr/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
pre_event_cat = Catalog.from_file(pre_event['stac:href'])
post_event_cat = Catalog.from_file(post_event['stac:href'])

In [39]:
pre_event_item = next(pre_event_cat.get_items())

pre_event_item.assets

{'nbr': <EOAsset href=./NBR_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndvi': <EOAsset href=./NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndwi': <EOAsset href=./NDWI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>}

In [40]:
post_event_item = next(post_event_cat.get_items())

post_event_item.assets

{'nbr': <EOAsset href=./NBR_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndvi': <EOAsset href=./NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndwi': <EOAsset href=./NDWI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>}

In [41]:
min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

In [42]:
post_event_item.assets['ndvi'].get_absolute_href()

'/workspace/nbr/stac-results/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif'

In [43]:
scaling_factor = 1/10000

scaling_factor

0.0001

If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [54]:
_mem = '/vsimem/mem.tif'

Process NDVI difference

In [54]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['ndvi'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

width = temp_ds.RasterXSize
height = temp_ds.RasterYSize
geo_transform = temp_ds.GetGeoTransform
geo_ref = temp_ds.GetProjectionRef()

pre_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [55]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['ndvi'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

post_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [58]:
delta_ndvi = ((pre_ndvi - post_ndvi) * scaling_factor).astype(float)

pre_ndvi = post_ndvi = None

0.0

Process NDWI difference

In [54]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['ndwi'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

ndvi_ds = None

In [55]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['ndwi'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [58]:
delta_ndwi = ((pre_ndwi - post_ndwi) * scaling_factor).astype(float)

pre_ndwi = pre_ndwi = None

0.0

Burned area delineation

In [ ]:
conditions = lambda delta_ndwi, ndwi_threshold, delta_ndvi, ndvi_threshold: 1 if ((delta_ndwi  > float(ndwi_threshold)) & (delta_ndvi > float(ndvi_threshold))) else 0
                             
vfunc_conditions = np.vectorize(conditions, otypes=[np.uint8])

In [ ]:
burned = vfunc_conditions(delta_ndwi,
                          ndwi_threshold['value'], 
                          delta_ndvi, 
                          ndvi_threshold['value']) 

relativized burn ratio (RBR) 

In [60]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['nbr'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

pre_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['nbr'].get_absolute_href(),
                         projWin=[min_lon, max_lat, max_lon, min_lat],
                         projWinSRS='EPSG:4326',
                         outputType=gdal.GDT_Int16)

post_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_nbr = ((pre_nbr  - post_nbr) * scaling_factor).astype(float)

post_nbr = None

In [ ]:
rbr = delta_nbr / (pre_nbr * scaling_factor + 1.001)

delta_nbr = pre_nbr = None

### Write results

In [24]:
def cog(input_tif, output_tif,no_data=None):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                    '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                    '-co COMPRESS=LZW '))
    
    if no_data != None:
        translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                        '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                        '-co COMPRESS=LZW '\
                                                                        '-a_nodata {}'.format(no_data)))
    ds = gdal.Open(input_tif, gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    
    ds = None

    ds = gdal.Open(input_tif)
    gdal.Translate(output_tif,
                   ds, 
                   options=translate_options)
    ds = None

    os.remove('{}.ovr'.format(input_tif))
    os.remove(input_tif)


In [25]:
catalog = Catalog(id='catalog', description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [26]:
output_name = 'DELINEATION'

In [27]:
result_item = EOItem(id=output_name,
                    geometry=item.geometry,
                    bbox=item.bbox,
                    datetime=item.datetime,
                    properties={},
                    bands=bands,
                    gsd=10, 
                    platform=item.platform, 
                    instrument=item.instrument)

In [28]:
for index, veg_index in enumerate(['NBR', 'NDVI', 'NDWI']):

    temp_name = '_{}_{}.tif'.format(veg_index, item.id)
    output_name = '{}_{}.tif'.format(veg_index, item.id)

    driver = gdal.GetDriverByName('GTiff')
    
    output = driver.Create(temp_name, 
                           width, 
                           height, 
                           1, 
                           gdal.GDT_Int16)

    output.SetGeoTransform(input_geotransform)
    output.SetProjection(input_georef)
    output.GetRasterBand(1).WriteArray(nbr),

    output.FlushCache()

    sleep(5)
    
    output = None

    del(output)

    cog(temp_name, output_name)

    

    result_item.add_asset(key=veg_index.lower(),
                          asset=EOAsset(href='./{}'.format(output_name), 
                          media_type=MediaType.GEOTIFF, 
                          title=bands[index]['name'],
                          bands=bands[index]))

In [29]:
catalog.add_items([result_item])

In [30]:
os.remove(tif)

In [31]:
catalog.normalize_and_save(root_href='stac-results',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [32]:
catalog.describe()

* <Catalog id=catalog>
  * <EOItem id=INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
